In [ ]:
# Imports
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# Images directories
train_dir = './train_images'
test_dir = './test_images'

# Normalize data -> prepare transformer
transform = transforms.Compose(
    [transforms.Grayscale(), 
     transforms.ToTensor(), 
     transforms.Normalize(mean=(0,),std=(1,))])

# Import data and transform them
train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

# Size of training / data -> 20% goes to validation
valid_size = 0.2

# When train ML, send batch by batch -> we send packages (batch) of 32 images every step
batch_size = 32

# Get index according to length, and shuffle it -> separate validation and training randomly every time
num_train = len(train_data)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)

# Get index used for validation by spliting array
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

# Sampler function
train_sampler = SubsetRandomSampler(train_new_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# What to feed to training
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=1)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=1)
classes = ('noface','face')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
# Network, ready to use
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2) #Pooling 2 by 2, window of 2
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Network creation
net = Net()

In [ ]:
# Calculate Entropy error -> test accuracy of model, we want criterion to tend to 0.
# Optimize change network each time to make it better. It's ran between every batches
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Model training

In [ ]:
saved_loss = []
for epoch in range(10):  # loop over the dataset multiple times -> epoch = nb of time we go through dataset, careful with overfitting

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward (calculating via current model) + backward (learning from mistakes) + optimize (fix mistakes)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item() # We want to minimize loss
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            if i == 2199:
                saved_loss.append(running_loss)
            running_loss = 0.0

print('Finished Training')

## Testing

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
inputs, classes = next(iter(test_loader))   

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
dataiter = iter(test_loader)
images, labels = next(dataiter)

image_tensor = torchvision.utils.make_grid(images[0])
image_label = labels[0]

# show images
imshow(image_tensor)


In [ ]:
labels[0]

### Printing loss according to epoch, to find best model and avoid overfitting

In [ ]:

# X axis parameter:
xaxis = range(0, 10)

# Y axis parameter:
yaxis = saved_loss 

plt.plot(xaxis, yaxis, 'ro')
plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)
plt.show()